In [156]:
import pandas as pd
import numpy as numpy 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

df = pd.read_csv("Account.csv")
df.usernameDigitCount = df["usernameDigitCount"].div(df["usernameLength"], axis=0)
df = df.drop(['usernameLength'], axis=1)
columns = df.columns

display(df)

,userFollowerCount,userFollowingCount,userBiographyLength,userMediaCount,userHasProfilPic,userIsPrivate,usernameDigitCount,isFake
0,258,238,0,0,1,0,0.000000,0
1,263,482,30,29,1,1,0.000000,0
2,51,78,9,0,1,1,0.000000,0
3,297,480,22,25,1,1,0.222222,0
4,113,242,0,95,1,1,0.000000,0
...,...,...,...,...,...,...,...,...
1189,0,25,138,6,1,0,0.000000,1
1190,0,0,52,6,1,0,0.428571,1
1191,0,1,92,9,1,0,0.066667,1
1192,1,0,60,5,1,0,0.076923,1


In [ ]:
for feature in columns[:-1]:
    plt.hist(df[df["isFake"] == 1][feature], color = 'red', label = 'Fake', alpha = 0.7, density = True)
    plt.hist(df[df["isFake"] == 0][feature], color = 'blue', label = 'Genuine', alpha = 0.7, density = True)
    plt.title(feature)
    plt.ylabel("Probability")
    plt.xlabel(feature)
    plt.legend()
    plt.show()

In [ ]:
#Correlation matrix



In [145]:
# train, valid, test split
#train, valid, test = numpy.split(df.sample(frac=1), [int(0.6 * len(df)), int(0.8 * len(df))])
X = df[df.columns[:-1]]
Y = df.isFake
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [147]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, confusion_matrix, ConfusionMatrixDisplay

random_forest = RandomForestClassifier()

def best_rf():
    model = random_forest.fit(x_train, y_train)
    old_f1Score = f1_score(model.predict(x_test), y_test)

    for i in range(25):
     param_dist = {'n_estimators': randint(50, 500), 
              'max_depth': randint(1, 20)}
     rand_search = RandomizedSearchCV(random_forest, param_distributions = param_dist, n_iter=5, cv=5)
     rand_search.fit(x_train, y_train)
    
     new_f1Score =  f1_score(rand_search.predict(x_test), y_test)

     if (new_f1Score >= old_f1Score):
        model = rand_search

    return model

In [148]:
best_model = best_rf()

best_model.fit(x_train, y_train)
y_pred = best_model.predict(x_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.97      0.96      0.96       199
           1       0.81      0.85      0.83        40

    accuracy                           0.94       239
   macro avg       0.89      0.90      0.90       239
weighted avg       0.94      0.94      0.94       239



In [158]:
import pickle
pickle_out = open("classifier.pkl", "wb")
pickle.dump(best_model, pickle_out)
pickle_out.close()